<a href="https://colab.research.google.com/github/ThomasTranGithub/Algo.Trading.Strategies/blob/main/RNN_Price_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import matplotlib as plt
import yfinance as yf
import seaborn as sn

A recurrent neural network (RNN) is a class of artificial neural networks where connections between nodes can create a cycle, allowing output from some nodes to affect subsequent input to the same nodes. This allows it to exhibit temporal dynamic behavior. Derived from feedforward neural networks, RNNs can use their internal state (memory) to process variable length sequences of inputs.[1][2][3] This makes them applicable to tasks such as unsegmented, connected handwriting recognition[4] or speech recognition.[5][6] Recurrent neural networks are theoretically Turing complete and can run arbitrary programs to process arbitrary sequences of inputs.[7]

import the prices for two holdings


we will 

In [15]:
import yfinance as yf

# Define the ticker symbol and time period
ticker_symbol = ['AAPL', 'GOOG']
start_date = "2022-01-01"
end_date = "2022-12-31"

# Fetch the stock data
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Print the stock data
print(stock_data)

[*********************100%***********************]  2 of 2 completed
             Adj Close                   Close                    High  \
                  AAPL        GOOG        AAPL        GOOG        AAPL   
Date                                                                     
2022-01-03  180.434296  145.074493  182.009995  145.074493  182.880005   
2022-01-04  178.144287  144.416504  179.699997  144.416504  182.940002   
2022-01-05  173.405685  137.653503  174.919998  137.653503  180.169998   
2022-01-06  170.510971  137.550995  172.000000  137.550995  175.300003   
2022-01-07  170.679474  137.004501  172.169998  137.004501  174.139999   
...                ...         ...         ...         ...         ...   
2022-12-23  131.477127   89.809998  131.860001   89.809998  132.419998   
2022-12-27  129.652435   87.930000  130.029999   87.930000  131.410004   
2022-12-28  125.674026   86.459999  126.040001   86.459999  131.029999   
2022-12-29  129.233658   88.949997  129.610

In [16]:
# Extract adjusted closing prices and volume for AAPL and GOOG
adj_close_prices = stock_data['Adj Close']
volume = stock_data['Volume']

# Print the extracted data
print("Adjusted Closing Prices:")
print(adj_close_prices)

print("\nVolume:")
print(volume)


Adjusted Closing Prices:
                  AAPL        GOOG
Date                              
2022-01-03  180.434296  145.074493
2022-01-04  178.144287  144.416504
2022-01-05  173.405685  137.653503
2022-01-06  170.510971  137.550995
2022-01-07  170.679474  137.004501
...                ...         ...
2022-12-23  131.477127   89.809998
2022-12-27  129.652435   87.930000
2022-12-28  125.674026   86.459999
2022-12-29  129.233658   88.949997
2022-12-30  129.552719   88.730003

[251 rows x 2 columns]

Volume:
                 AAPL      GOOG
Date                           
2022-01-03  104487900  25214000
2022-01-04   99310400  22928000
2022-01-05   94537600  49642000
2022-01-06   96904000  29050000
2022-01-07   86709100  19408000
...               ...       ...
2022-12-23   63814900  17815000
2022-12-27   69007800  15470900
2022-12-28   85438400  17879600
2022-12-29   75703700  18280700
2022-12-30   77034200  19190300

[251 rows x 2 columns]


To use the volume and closing price data as inputs for an RNN (Recurrent Neural Network) model, you can follow these steps:

Import the necessary libraries:

In [27]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

Prepare the input data:

In [28]:
# Assuming you have extracted the volume and adjusted closing prices
volume = stock_data['Volume'].values
closing_price = stock_data['Adj Close'].values

# Normalize the input data
normalized_volume = (volume - np.mean(volume)) / np.std(volume)
normalized_closing_price = (closing_price - np.mean(closing_price)) / np.std(closing_price)

# Combine the normalized volume and closing price into a single input array
input_data = np.column_stack((normalized_volume, normalized_closing_price))


In [29]:
# Assuming you have a target variable indicating the desired prediction
# For example, if you want to predict whether the closing price will increase or decrease the next day:
target = np.where(closing_price[1:] > closing_price[:-1], 1, 0)  # 1: increase, 0: decrease

# Reshape the target array to match the input shape
target = target.reshape(-1, 1)


Prepare the target data:

In [30]:
# Assuming you have defined the training and testing set sizes
train_size = int(0.8 * len(input_data))

x_train = input_data[:train_size]
y_train = target[:train_size]

x_test = input_data[train_size:]
y_test = target[train_size:]

In [32]:
x_train = x_train.reshape(-1, 2, 1)


Build and train the RNN model:

In [33]:
model = Sequential()
model.add(LSTM(64, input_shape=(2, 1)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, batch_size=32)




ValueError: ignored